## Setup

In [1]:
import json
import numpy as np
import os
import random
import sys
import torch
sys.path.append("../..")

from evals.spelling_by_grade import create_full_spelling_prompt, prepare_grade_spelling_eval
from evals.plot_utils import basic_bar_graph, create_table, nested_bar_graph
from evals.eval_utils import ModelType
from evals.eval_list import *
from transformer_lens import HookedTransformer, utils

import plotly.io as pio
pio.renderers.default = "iframe"

words_by_grade = prepare_grade_spelling_eval("../data/GradeSpellingEval.txt", ' ')
words_by_grade[3][:10]

[('able', 'A B L E'),
 ('above', 'A B O V E'),
 ('afraid', 'A F R A I D'),
 ('afternoon', 'A F T E R N O O N'),
 ('again', 'A G A I N'),
 ('age', 'A G E'),
 ('air', 'A I R'),
 ('airplane', 'A I R P L A N E'),
 ('almost', 'A L M O S T'),
 ('alone', 'A L O N E')]

In [2]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.set_grad_enabled(False)

model = HookedTransformer.from_pretrained("pythia-2.8b")
tokenizer = model.tokenizer

n_layers = model.cfg.n_layers
d_model = model.cfg.d_model
n_heads = model.cfg.n_heads
d_head = model.cfg.d_head
d_mlp = model.cfg.d_mlp
d_vocab = model.cfg.d_vocab

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


Loaded pretrained model pythia-2.8b into HookedTransformer


In [3]:
example_prompt = \
"""
Cat: C A T
Dog: D O G
Bird: B I R D
Fish: F I S H
Apple:"""
example_answer = " A"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True, prepend_space_to_answer=False)

Tokenized prompt: ['<|endoftext|>', '\n', 'Cat', ':', ' C', ' A', ' T', '\n', 'Dog', ':', ' D', ' O', ' G', '\n', 'B', 'ird', ':', ' B', ' I', ' R', ' D', '\n', 'Fish', ':', ' F', ' I', ' S', ' H', '\n', 'Apple', ':']
Tokenized answer: [' A']


Performance on answer token:
Rank: 0        Logit: 21.78 Prob: 96.88% Token: | A|

Top 0th token. Logit: 21.78 Prob: 96.88% Token: | A|
Top 1th token. Logit: 16.23 Prob:  0.38% Token: | a|
Top 2th token. Logit: 15.72 Prob:  0.23% Token: |  |
Top 3th token. Logit: 15.55 Prob:  0.19% Token: | I|
Top 4th token. Logit: 15.45 Prob:  0.17% Token: | S|
Top 5th token. Logit: 15.41 Prob:  0.17% Token: | O|
Top 6th token. Logit: 15.24 Prob:  0.14% Token: | P|
Top 7th token. Logit: 15.15 Prob:  0.13% Token: | Apple|
Top 8th token. Logit: 15.03 Prob:  0.11% Token: | H|
Top 9th token. Logit: 14.90 Prob:  0.10% Token: | M|


Ranks of the answer tokens: [(' A', 0)]

## DLA

In [4]:
single_shot_eval = GET_FIRST_LETTER_EVAL.run_eval(model, ModelType.TRANSFORMER_LENS, tokenizer, words_by_grade, 5)

Assessing Grade 1


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:04,  1.81it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

 50%|█████     | 2/4 [00:01<00:01,  1.83it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

 75%|███████▌  | 3/4 [00:01<00:00,  1.82it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:02<00:00,  1.87it/s]


In [6]:
prompts = []
for grade in single_shot_eval['data'].keys():
    for item in single_shot_eval['data'][grade]:
        if len(item['word']) > 1:
            prompts.append(item)

len(prompts)

846

In [7]:
import tqdm

# a_proj, e_proj, a_e_proj = torch.zeros(len(prompts), 1058), torch.zeros(len(prompts), 1058), torch.zeros(len(prompts), 1058)

for i, prompt in tqdm.tqdm(enumerate(prompts)):
    logits, cache = model.run_with_cache(prompt['prompt'], prepend_bos=True)
    a_token = model.to_single_token(f" {prompt['word'][0].upper()}")
    e_token = model.to_single_token(f" {prompt['word'][1].upper()}")
    
    a_unembed = model.W_U[:, a_token].detach().cpu()
    e_unembed = model.W_U[:, e_token].detach().cpu()
    
    dif_a_e = a_unembed - e_unembed
    decomp, labels = cache.get_full_resid_decomposition(expand_neurons=False, return_labels=True)
    decomp = decomp[:,0,-1].detach().cpu()

    cur_a, cur_e, cur_a_e = decomp @ a_unembed, decomp @ e_unembed, decomp @ dif_a_e
    # a_proj[i] = decomp @ a_unembed
    # e_proj[i] = decomp @ e_unembed
    # a_e_proj[i] = decomp @ dif_a_e
    
    if i == 0:
        a_proj, e_proj, a_e_proj = cur_a, cur_e, cur_a_e
    else:
        # Keep a rolling average as I go. E.g, 6th item = current * 5/6 + new * 1/6.
        a_proj = a_proj * (i / (i+1)) + (1 / (i+1)) * cur_a
        e_proj = e_proj * (i / (i+1)) + (1 / (i+1)) * cur_e
        a_e_proj = a_e_proj * (i / (i+1)) + (1 / (i+1)) * cur_a_e

1it [00:00,  3.56it/s]

Tried to stack head results when they weren't cached. Computing head results now


2it [00:00,  3.45it/s]

Tried to stack head results when they weren't cached. Computing head results now


4it [00:00,  4.66it/s]

...

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


844it [02:16,  6.92it/s]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


846it [02:17,  6.17it/s]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


In [8]:
# print(a_proj.shape, e_proj.shape, a_e_proj.shape)
# a_proj, e_proj, a_e_proj = torch.sum(a_proj, 0) / len(prompts), torch.sum(e_proj, 0) / len(prompts), torch.sum(a_e_proj, 0) / len(prompts)
# print(a_proj.shape, e_proj.shape, a_e_proj.shape)

In [9]:
import pandas as pd
import plotly.express as px

decomp_df = pd.DataFrame({
    "labels": labels,
    "a_proj": a_proj,
    "e_proj": e_proj,
    "a_e_proj": a_e_proj,}
)

px.line(decomp_df, x="labels", y=["a_proj", "e_proj", "a_e_proj"], title="Projection of Residuals onto A and E Embeddings")

In [10]:
top_indices = torch.topk(a_e_proj, 10)
print(top_indices)
top_labels = [labels[i] for i in top_indices.indices]
print([(round(top_indices.values[i].item(), 3), labels[top_indices.indices[i]]) for i in range(len(top_labels))])

torch.return_types.topk(
values=tensor([7.9888, 5.6485, 2.8568, 1.4510, 0.8683, 0.7782, 0.6587, 0.6102, 0.5859,
        0.5843]),
indices=tensor([ 575,  517,  913,  553,  424, 1054,  430, 1046,  964, 1050]))
[(7.989, 'L17H31'), (5.649, 'L16H5'), (2.857, 'L28H17'), (1.451, 'L17H9'), (0.868, 'L13H8'), (0.778, '30_mlp_out'), (0.659, 'L13H14'), (0.61, '22_mlp_out'), (0.586, 'L30H4'), (0.584, '26_mlp_out')]


In [11]:
logits, cache = model.run_with_cache(example_prompt, prepend_bos=True)
results, labels = cache.accumulated_resid(apply_ln=True, return_labels=True, mlp_input=False, incl_mid=False)
results.shape

torch.Size([33, 1, 31, 2560])

In [22]:
# DLA per layer
logits, cache = model.run_with_cache(example_prompt, prepend_bos=True)
a_token = model.to_single_token(" A")
e_token = model.to_single_token(" P")

a_unembed = model.W_U[:, a_token].detach().cpu()
e_unembed = model.W_U[:, e_token].detach().cpu()

dif_a_e = a_unembed - e_unembed
decomp, labels = cache.accumulated_resid(apply_ln=True, return_labels=True, mlp_input=False, incl_mid=False)
decomp = decomp[:,0,-1].detach().cpu()

cur_a, cur_e, cur_a_e = decomp @ a_unembed, decomp @ e_unembed, decomp @ dif_a_e
print(cur_a.shape, cur_e.shape, cur_a_e.shape)

torch.Size([33]) torch.Size([33]) torch.Size([33])


In [23]:
print(model.to_str_tokens(example_prompt))
print(model.to_string(a_token), model.to_string(e_token))

['<|endoftext|>', '\n', 'Cat', ':', ' C', ' A', ' T', '\n', 'Dog', ':', ' D', ' O', ' G', '\n', 'B', 'ird', ':', ' B', ' I', ' R', ' D', '\n', 'Fish', ':', ' F', ' I', ' S', ' H', '\n', 'Apple', ':']
 A  P


In [24]:
decomp_df = pd.DataFrame({
    "labels": labels,
    "a_proj": cur_a,
    "e_proj": cur_e,
    "a_e_proj": cur_a_e,}
)

px.line(decomp_df, x="labels", y=["a_proj", "e_proj", "a_e_proj"], title="Projection of Residuals onto A and E Embeddings")

In [30]:
top_indices = torch.topk(a_proj, 10)
decomp, labels = cache.get_full_resid_decomposition(expand_neurons=False, return_labels=True)
print(top_indices)
top_labels = [labels[i] for i in top_indices.indices]
print([(round(top_indices.values[i].item(), 3), labels[top_indices.indices[i]]) for i in range(len(top_labels))])

Tried to stack head results when they weren't cached. Computing head results now
torch.return_types.topk(
values=tensor([10.2988,  9.2339,  8.7282,  7.7743,  6.7722,  5.5483,  5.2606,  5.1224,
         4.1601,  3.6914]),
indices=tensor([1055, 1048,  575, 1057,  517, 1044,  913, 1046, 1045, 1047]))
[(10.299, '31_mlp_out'), (9.234, '24_mlp_out'), (8.728, 'L17H31'), (7.774, 'bias'), (6.772, 'L16H5'), (5.548, '20_mlp_out'), (5.261, 'L28H17'), (5.122, '22_mlp_out'), (4.16, '21_mlp_out'), (3.691, '23_mlp_out')]
